### Checkpoints for hubNashville project:
- a.  Thursday afternoon (checkpoint 1): what questions would teams like to answer (we'll create assignment repository at this point)
- b.  Monday afternoon (checkpoint 2): preliminary findings/what to focus on to add to presentation
- c.  Tuesday afternoon (final checkpoint): presentation walkthrough

#### TEAM CHEETAH (AMANDA AND PAM)
- Look at differences (who is calling and why, % of calls) between requests online (website and app) and phone calls.

- Identify differences (time of day/month/year, location, request type, etc.) to focus marketing to direct more users - to online platforms

- Are there certain types that lend themselves to reporting through phone, use this as an opportunity to improve or modify online experience for these request types

#### ORIGINAL LIST OF IDEAS
- `Date/Time` columns need to change to date/time? - mike did this, plus date difference column
- Change `Mapped Location` to geo point
- `Council District` needs to change to integer
- Do we need to keep the nulls? - mike got rid of nulls in certain fields
- May need to split date/time, do we need to keep time?
- What time of day are most requests opened/closed?
- What month has the most requests opened/closed?
- How long Open requests have been open, how long have the In Progress have been in progress?
- Which cities or zips generate the most requests? (make a map with popup showing type of request)
- How many requests were `closed when created`? QUICK REQUESTS
- What types of questions come in from the mayor's office as opposed to another `request origin`?
- Resident calls (much more) vs. business calls
- Would it be beneficial to find out what the "Other" is for `Contact Type`? Might be able to add more specific types.
- Explore parent requests and see if related to # of child requests or other metric (time taken to close).
- Types of request that are most called on (trash) - find keywords to break "other" down
- for subrequest = quick, request type = quick, no further info -- we recommend use of categories for better description
- ** to help move folks to online platforms, look at areas and types that are not currently on online platform -- don't look at hub and mobile app, and see what types of requests, who is calling and why, % of calls
- Question - Appendix A of the PDF only shows 5 status types, but the value counts shows 10 types.  Do we want to drop the types that are not listed on the Appendix?

In [2]:
from shapely.geometry import Point
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [3]:
servreq = pd.read_csv('../data/hubNashville__311__Service_Requests.csv', low_memory=False)
print(servreq.shape)
servreq.head()
servreq.dtypes

(348671, 21)


Request #                       int64
Status                         object
Request Type                   object
Subrequest Type                object
Additional Subrequest Type     object
Date / Time Opened             object
Date / Time Closed             object
Request Origin                 object
System of Origin ID            object
Contact Type                   object
Preferred Language            float64
State Issue                      bool
Parent Request                float64
Closed When Created              bool
Address                        object
City                           object
Council District              float64
ZIP                            object
Latitude                      float64
Longitude                     float64
Mapped Location                object
dtype: object

In [4]:
servreq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348671 entries, 0 to 348670
Data columns (total 21 columns):
Request #                     348671 non-null int64
Status                        348671 non-null object
Request Type                  348603 non-null object
Subrequest Type               338024 non-null object
Additional Subrequest Type    180510 non-null object
Date / Time Opened            348671 non-null object
Date / Time Closed            338424 non-null object
Request Origin                348671 non-null object
System of Origin ID           26398 non-null object
Contact Type                  26129 non-null object
Preferred Language            0 non-null float64
State Issue                   348671 non-null bool
Parent Request                3589 non-null float64
Closed When Created           348671 non-null bool
Address                       285058 non-null object
City                          280771 non-null object
Council District              280574 non-null float64

In [5]:
servreq.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,113016,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/30/2018 01:47:00 PM,03/05/2019 09:35:00 PM,Phone,NaN,NaN,...,False,NaN,False,2009 Upland Dr,NASHVILLE,7.0,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,126185,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,NaN,12/28/2018 01:04:00 PM,12/28/2018 01:04:00 PM,Phone,OE-00040610_12/27/2018 15:44,NaN,...,False,NaN,True,ELM HILL PKE / DONELSON PKE,NASHVILLE,15.0,37214,36.148809,-86.666282,"(36.14880891, -86.6662816)"
2,119344,Closed,"Trash, Recycling & Litter",Damage to Property,Trash Service,11/27/2018 02:20:00 PM,11/29/2018 04:04:00 AM,Phone,NaN,NaN,...,False,NaN,True,"597 Westchester Dr, Nashville, TN 37115, Unite...",MADISON,8.0,37115,36.263020,-86.738630,"(36.26302, -86.73863)"
3,127752,Closed,"Trash, Recycling & Litter",New Service/Activate Service,New Construction Residential Recycle,01/03/2019 06:57:00 PM,01/09/2019 10:09:00 AM,Phone,NaN,NaN,...,False,NaN,False,Ruffian Way,NaN,NaN,NaN,38.097609,-84.554097,"(38.0976091, -84.5540971)"
4,113477,Closed,"Trash, Recycling & Litter",Backdoor Trash or Recycle Waivers,Request Recycling Disability Waiver,10/31/2018 12:20:00 PM,10/31/2018 04:10:00 PM,Phone,NaN,NaN,...,False,NaN,True,"2326 Smith Springs Rd, Nashville, TN 37217, Un...",NASHVILLE,29.0,37217,36.096460,-86.637440,"(36.09646, -86.63744)"


In [7]:
req_online = servreq[servreq['Request Origin'].isin(['Mobile App', 'hubNashville Community'])]
req_online.head(5)

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
5,127499,Closed,"Streets, Roads & Sidewalks",Request New Signal,NaN,01/03/2019 07:39:00 AM,01/22/2019 09:01:00 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,Blue Hole Rd & Antioch Pike,ANTIOCH,32.0,37013,36.059970,-86.672119,"(36.05997021, -86.6721194)"
6,128356,Closed,"Trash, Recycling & Litter",Cart Service,Recycling - Request an Additional Cart,01/05/2019 03:30:00 AM,01/17/2019 10:00:00 AM,hubNashville Community,NaN,NaN,...,False,NaN,False,1513A Wendell Ave,NASHVILLE,7.0,37206,36.193906,-86.710023,"(36.1939065, -86.7100229)"
15,114597,Closed,"Streets, Roads & Sidewalks",Roadwork Complaint,NaN,11/05/2018 04:38:00 PM,11/20/2018 11:13:00 AM,hubNashville Community,NaN,NaN,...,False,NaN,True,"20 Music Cir E, Nashville, TN 37203, United St...",NASHVILLE,19.0,37203,36.151670,-86.789510,"(36.15167, -86.78951)"
22,323738,Closed,Other,Mayor's Office Case,NaN,03/10/2020 05:07:20 AM,03/11/2020 07:41:37 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,127348,Closed,Property Violations,Junk and Debris,NaN,01/02/2019 07:43:00 PM,03/12/2019 08:03:00 AM,hubNashville Community,NaN,NaN,...,False,123149.0,False,5128 Hilson Rd,NASHVILLE,27.0,37211,36.053315,-86.727140,"(36.05331531, -86.72714015)"


In [6]:
servreq_counts = servreq.groupby(['Council District', 'Request Origin']).size().to_frame().reset_index()
servreq_counts.head(10)

,Council District,Request Origin,0
0,1.0,Email,21
1,1.0,Mayor's Office,18
2,1.0,Mobile App,111
3,1.0,Phone,5780
4,1.0,Walk-In,1
5,1.0,hubNashville Community,1175
6,2.0,Council Member,1
7,2.0,Council's Website,1
8,2.0,Email,50
9,2.0,Mayor's Office,11


## Looking at value counts of Request Origin column

In [ ]:
print(servreq['Request Origin'].value_counts())
print(type(servreq['Request Origin']))

In [ ]:
print(servreq['Status'].value_counts())
print(type(servreq['Status']))

In [ ]:
request_percents = servreq['Request Origin'].value_counts(normalize=True).to_frame().reset_index()
request_percents.columns = ['Request Origin Type', 'Percent of Total Requests']
request_percents

In [ ]:
request_percents.plot(x='Request Origin Type',
                      y='Percent of Total Requests',
                      kind='barh')
plt.gca().invert_yaxis()
plt.title('Percent of Total Requests by Origin')
plt.xlabel('Percent of Total Requests')
plt.ylabel('Request Origin Type');
plt.show()

In [ ]:
subset_app_hub_reqs = servreq.query('Request Origin == ['Mobile App','hubNashville Community'])
subset_app_hub_reqs

In [ ]:
print(servreq['Request Type'].value_counts())
print(type(servreq['Request Type']))

In [ ]:
print(servreq['Subrequest Type'].value_counts())
print(type(servreq['Subrequest Type']))

## Looking at value counts of Contact Type column

In [ ]:
print(servreq['Contact Type'].value_counts())
print(type(servreq['Contact Type']))

#### Contact Type=Other seems to be mostly Request Type of Streets/Roads/Sidewalks.  Not sure what "other" is, but not listed as resident or business

In [ ]:
contact_other = servreq.loc[servreq['Contact Type'] == "Other"]
contact_other.head()

In [ ]:
#use this query as example to subset
#mayor_req_types_df = mayors_office_calls.query([mayors_office_calls['Request Type'])

Convert date/time fields from object to date/time, then calculate how long a request has been open in a new column, and move the column up to be close to original date/time fields. (code from Mike)

In [ ]:
t1 = pd.to_datetime(servreq['Date / Time Closed'])
t2 = pd.to_datetime(servreq['Date / Time Opened'])
servreq['time_taken'] = t1-t2

# move time_taken column to after time_closed

col_name = 'time_taken'
hub_time = servreq.pop(col_name)
servreq.insert(7, col_name, hub_time)
servreq.head()

Convert opened/closed columns back to datetime format

In [8]:
servreq["Date / Time Opened"]= pd.to_datetime(servreq["Date / Time Opened"])
servreq["Date / Time Closed"]= pd.to_datetime(servreq["Date / Time Closed"])
servreq.dtypes

Request #                              int64
Status                                object
Request Type                          object
Subrequest Type                       object
Additional Subrequest Type            object
Date / Time Opened            datetime64[ns]
Date / Time Closed            datetime64[ns]
Request Origin                        object
System of Origin ID                   object
Contact Type                          object
Preferred Language                   float64
State Issue                             bool
Parent Request                       float64
Closed When Created                     bool
Address                               object
City                                  object
Council District                     float64
ZIP                                   object
Latitude                             float64
Longitude                            float64
Mapped Location                       object
dtype: object

Break Date/Time Opened to separate year and month

In [ ]:
servreq['date_open'] = pd.to_datetime(servreq['Date / Time Opened'])
servreq['year_open'], servreq['monthnum_open'] = servreq['Date / Time Opened'].dt.year,servreq['Date / Time Opened'].dt.month
servreq.head(1)

Break Date/Time Closed to separate year and month

In [ ]:
servreq['date_closed'] = pd.to_datetime(servreq['Date / Time Closed'])
servreq['year_closed'], servreq['monthnum_closed'] = servreq['Date / Time Closed'].dt.year,servreq['Date / Time Closed'].dt.month
servreq.head(1)

In [ ]:
servreq['monthnum_open'].value_counts()

In [ ]:
servreq.sort_values(by = 'monthnum_open', ascending = True).head(1)

### take a look at `Council District` info

In [ ]:
servreq['Council District'].value_counts()

In [ ]:
median_district = servreq['Council District'].describe()
print(median_district)

In [ ]:
district_19 = servreq.loc[servreq['Council District'] == 19.0].head()
print(district_19.shape)
district_19.head()
# mask_2009 = gdp_and_internet_use_df['Year'] == 2009
# mask_2009_df = gdp_and_internet_use_df [mask_2009]
# print(mask_2009_df.head())

In [ ]:
councildist_df = gpd.read_file('../data/councildist_df.geojson')
print(councildist_df.crs)
councildist_df.head()

In [ ]:
councildist_df.crs = "EPSG:4326"
print(councildist_df.crs)
councildist_df.head()

In [ ]:
councildist_df = councildist_df[['council_district', 'geometry']]
#councildist_df.head(1)

In [ ]:
map_nulls_cd = servreq['Council District'].isnull()
map_nulls_cd.value_counts()

In [ ]:
leg_kwds = {'title': 'Nashville Metro Council Districts', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1), 'ncol': 3}

councildist_df.plot(column = 'council_district', figsize=(10, 10),
                edgecolor = 'black',
                legend = True, legend_kwds = leg_kwds,
                cmap = 'Set2_r')
plt.grid()
plt.title('Nashville Metro Council Districts')
plt.show()

In [ ]:
# cat's code for map, change to req_online

# covid_by_zip = covid_19_geo.groupby(['ZIP']).count()
# covid_by_zip = covid_by_zip.reset_index()
# covid_by_zip = covid_by_zip[['ZIP', 'Request Type']]
# covid_by_zip.head()
# merge_by_zip = pd.merge(zipcodes, covid_by_zip, left_on = 'zip', right_on = 'ZIP')
# map_nash_cov = folium.Map(location = [36.1227, -86.8269] , zoom_start = 10)
# marker_cluster = MarkerCluster().add_to(map_nash_cov)
# folium.GeoJson(zipcodes).add_to(map_nash_cov)
# #iterate through covid_19_geo to create locations and markers for each covid-related request
# for row_index, row_values in covid_19_geo.iterrows():
#     loc = [row_values['Latitude'], row_values['Longitude']]
#     pop = str(row_values['Subrequest Type'])
#     marker = folium.Marker(
#         location = loc,
#         popup = pop)
#     marker.add_to(marker_cluster)
# map_nash_cov.save('../maps/map_nash_cov.html')
# #display our map
# map_nash_cov

#### (NOT PURSUING THIS LINE OF ANALYSIS CURRENTLY) Looking at Request Origin of Mayor's Office only

In [ ]:
mayors_office_calls = servreq.loc[servreq['Request Origin'] == "Mayor's Office"]
mayors_office_calls.head()
#mayors_office_calls.tail()

#### (NOT PURSUING THIS LINE OF ANALYSIS CURRENTLY) Found no open calls to mayor's office

In [ ]:
mayor_calls_open = mayors_office_calls.query('Status == False')
mayor_calls_open.head()

#### (NOT PURSUING THIS LINE OF ANALYSIS CURRENTLY) trying to pull list of what types of requests from mayors_office_calls

In [ ]:
mayors_office_calls['Request Type'].value_counts()

#### Nearly 25% of the Council District and City columns are Nan or empty 
### USE FILLNA to fill the nan/nulls with spaces:
df.column = df.column.fillna(' ')
### OR fix apostrophes or other string issues with:
df.column = df.column.str.replace("'", "`")


In [ ]:
map_nulls_city = servreq['City'].isnull()
map_nulls_city.value_counts()
#map_nulls_city.head(10)